In [1]:
#Import Libraries
import csv
import pandas as pd
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions

In [3]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:   
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet   

In [10]:
# application variables
user = 'rdmmaqueensuni'
search_term = 'tesla'
my_password = 'kipling@7'

# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys(user)

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(1)

In [4]:
# find search input and search for term
search_term = 'tesla since:2020-10-09 until:2020-10-10'
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)

In [7]:
# navigate to historical 'latest' tab
driver.find_element_by_link_text('Latest').click()

# get all tweets on the page
data = []
data_temp = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

In [ ]:
tweet_month = ['tesla since:2020-10-09 until:2020-10-10','tesla since:2020-10-08 until:2020-10-09']
data = []

for day in tweet_month: 

    # find search input and search for term
    search_term = day
    search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)
    
    sleep(50)
    driver.find_element_by_link_text('Latest').click()
#     driver.find_element_by_css_selector('div.css-4rbku5 css-18t94o4 css-1dbjc4n r-1awozwy r-18p3no4 r-rull8r r-wgabs5 r-1loqt21 r-6koalj r-eqz5dr r-16y2uox r-1777fci r-1ny4l3l r-1oqcu8e r-o7ynqc r-6416eg').click()

    data_temp = []
    tweet_ids = set()
    scrolling = True
    last_position = driver.execute_script("return window.pageYOffset;")

    while scrolling:
        page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
        for card in page_cards[-15:]:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data_temp.append(tweet)

        scroll_attempt = 0
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(1)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2) # attempt another scroll
            else:
                last_position = curr_position
                break
    
    data.append(data_temp)
    
    # close the web driver
# driver.close()

In [6]:
data[:2]

[('10.13',
  '@bgeeezy__',
  '2020-10-10T20:35:02.000Z',
  'If I see a Tesla in the buckhead area, I’m automatically looking for \n@dearra\n ',
  '',
  '',
  ''),
 ('Rüya Öksin',
  '@tatli_cadi84',
  '2020-10-10T20:34:58.000Z',
  'Replying to \n@ama_tesla\n @bagsbanny34\n and \n@onediocomSesli gulduk',
  '',
  '',
  '')]

In [ ]:
print(len(data[0]))

In [7]:
with open('tesla_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [15]:
df = pd.DataFrame(data, columns =['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']) 
  
df.head()

,UserName,Handle,Timestamp,Comments,Likes,Retweets,Text
0,Rafael Torres Mejìa,@torrespld01,2020-10-10T21:01:09.000Z,Competencia para la #cybertruck de #Tesla :Quo...,,,
1,StatsApp,@StatsTeslaApp,2020-10-10T21:01:08.000Z,Replying to \n@chris_grnwld\n @DaddyFrunk\n an...,,,
2,la Jolla boy,@CpheinHein,2020-10-10T21:01:05.000Z,Replying to \n@droid_tesla\n @BarkMSmeagol\n a...,,,
3,The Shah of Blah,@onefumi,2020-10-10T21:00:59.000Z,"Replying to \n@tanyajunghansLike, he didn't ev...",,,
4,Drew,@DrewDamnjanovic,2020-10-10T21:00:55.000Z,Replying to \n@MemberSeeShareholders sell shar...,,,


In [16]:
df.to_csv(r"C:\Users\Tom-T\Google Drive\Queen's MMA\Analytics for Financial Markets - MMA 823\Project\Tesla_Tweets.csv")